### 1. Requirement

In [1]:
import pennylane as qml
from pennylane import qchem
from pennylane import numpy as np

A_to_au_conversion = 1.8897259885789

### 2. Define Function

In [3]:
def calc_adapt_vqe_LiH(r, E, active_electrons=2, active_orbitals=5):
    symbols = ["Li", "H"]
    geometry = np.array([0.0, 0.0, 0.0, 0.0, 0.0, r*A_to_au_conversion])

    H, qubits = qchem.molecular_hamiltonian(
        symbols,
        geometry,
        active_electrons=active_electrons,
        active_orbitals=active_orbitals
    )
    print(H)

    active_electrons = active_electrons

    singles, doubles = qchem.excitations(active_electrons, qubits)

    print(f"Total number of excitations = {len(singles) + len(doubles)}")
    singles_excitations = [qml.SingleExcitation(0.0, x) for x in singles]
    doubles_excitations = [qml.DoubleExcitation(0.0, x) for x in doubles]
    operator_pool = doubles_excitations + singles_excitations   
    hf_state = qchem.hf_state(active_electrons, qubits)
    dev = qml.device("default.qubit", wires=qubits)
    @qml.qnode(dev)
    def circuit():
        [qml.PauliX(i) for i in np.nonzero(hf_state)[0]]
        return qml.expval(H)
    energy_array = []
    opt = qml.optimize.AdaptiveOptimizer()
    for i in range(len(operator_pool)):
        circuit, energy, gradient = opt.step_and_cost(circuit, operator_pool)
        energy_array.append(energy)
        if i % 3 == 0:
            print("n = {:},  E = {:.8f} H, Largest Gradient = {:.3f}".format(i, energy, gradient))
            # print(qml.draw(circuit, decimals=None)())
            print()
        if gradient < E*10^(-E):
            break
    return energy_array, circuit


In [5]:
calc_adapt_vqe_LiH(1.57, 3)

  (-5.727145625841578) [I0]
+ (-0.5679788236408931) [Z8]
+ (-0.5679788236408931) [Z9]
+ (-0.4257001406919125) [Z6]
+ (-0.4257001406919125) [Z7]
+ (-0.42570014069174167) [Z4]
+ (-0.42570014069174167) [Z5]
+ (-0.39037445148575484) [Z2]
+ (-0.39037445148575484) [Z3]
+ (-0.2992504685419076) [Z1]
+ (-0.29925046854190757) [Z0]
+ (-0.0017720690477482687) [Y1 Y3]
+ (-0.0017720690477482687) [X1 X3]
+ (0.012018914489065034) [Y0 Y2]
+ (0.012018914489065034) [X0 X2]
+ (0.05292758391024259) [Z0 Z2]
+ (0.05292758391024259) [Z1 Z3]
+ (0.053809640736503996) [Z2 Z8]
+ (0.053809640736503996) [Z3 Z9]
+ (0.05612007385422833) [Z0 Z3]
+ (0.05612007385422833) [Z1 Z2]
+ (0.06019487899991935) [Z2 Z4]
+ (0.06019487899991935) [Z3 Z5]
+ (0.06019487899994722) [Z2 Z6]
+ (0.06019487899994722) [Z3 Z7]
+ (0.06040619664741653) [Z2 Z9]
+ (0.06040619664741653) [Z3 Z8]
+ (0.06201646674715919) [Z0 Z4]
+ (0.06201646674715919) [Z1 Z5]
+ (0.06201646674718789) [Z0 Z6]
+ (0.06201646674718789) [Z1 Z7]
+ (0.06221195298318677) [Z4

([tensor(-7.86269496, requires_grad=True),
  tensor(-7.87690671, requires_grad=True),
  tensor(-7.87870889, requires_grad=True),
  tensor(-7.88039443, requires_grad=True),
  tensor(-7.88096931, requires_grad=True),
  tensor(-7.88151971, requires_grad=True),
  tensor(-7.88190175, requires_grad=True),
  tensor(-7.88195155, requires_grad=True),
  tensor(-7.88197563, requires_grad=True),
  tensor(-7.88200802, requires_grad=True),
  tensor(-7.88208619, requires_grad=True),
  tensor(-7.88215831, requires_grad=True),
  tensor(-7.88218805, requires_grad=True),
  tensor(-7.88222662, requires_grad=True),
  tensor(-7.88224253, requires_grad=True),
  tensor(-7.88227584, requires_grad=True),
  tensor(-7.88228971, requires_grad=True),
  tensor(-7.88231505, requires_grad=True),
  tensor(-7.8823369, requires_grad=True),
  tensor(-7.88234565, requires_grad=True),
  tensor(-7.88235285, requires_grad=True),
  tensor(-7.88236051, requires_grad=True),
  tensor(-7.88237428, requires_grad=True),
  tensor(-7.

### 3. Create Range

In [3]:
r_array = np.arange(0.5, 5.0, 0.25)
r_array

tensor([0.5 , 0.75, 1.  , 1.25, 1.5 , 1.75, 2.  , 2.25, 2.5 , 2.75, 3.  ,
        3.25, 3.5 , 3.75, 4.  , 4.25, 4.5 , 4.75], requires_grad=True)

### 4. Calculate Energies 

In [4]:
E_R = [] # saved in E_R, contains E in each R
circuits = []
threshold = 3

for i in range(len(r_array)):
    print(i)
    E, circuit = calc_adapt_vqe_LiH(r_array[i], threshold, active_electrons=4)
    circuits.append(circuit)
    E_R.append(E[-1])

0
Total number of excitations = 54
n = 0,  E = -7.02840997 H, Largest Gradient = 0.040

n = 3,  E = -7.03659619 H, Largest Gradient = 0.014

n = 6,  E = -7.03712489 H, Largest Gradient = 0.008

n = 9,  E = -7.03719621 H, Largest Gradient = 0.007

n = 12,  E = -7.03732084 H, Largest Gradient = 0.006

n = 15,  E = -7.03740454 H, Largest Gradient = 0.005

n = 18,  E = -7.03767054 H, Largest Gradient = 0.004

n = 21,  E = -7.03788047 H, Largest Gradient = 0.004

n = 24,  E = -7.03801585 H, Largest Gradient = 0.003

n = 27,  E = -7.03810394 H, Largest Gradient = 0.003

n = 30,  E = -7.03820701 H, Largest Gradient = 0.002

n = 33,  E = -7.03826650 H, Largest Gradient = 0.002

n = 36,  E = -7.03830534 H, Largest Gradient = 0.002

n = 39,  E = -7.03834198 H, Largest Gradient = 0.002

n = 42,  E = -7.03836364 H, Largest Gradient = 0.002

n = 45,  E = -7.03838514 H, Largest Gradient = 0.001

n = 48,  E = -7.03840128 H, Largest Gradient = 0.001

n = 51,  E = -7.03841572 H, Largest Gradient = 0.00

In [9]:
E_R

[tensor(-7.03842279, requires_grad=True),
 tensor(-7.56018936, requires_grad=True),
 tensor(-7.76988249, requires_grad=True),
 tensor(-7.8471033, requires_grad=True),
 tensor(-7.86540146, requires_grad=True),
 tensor(-7.85585543, requires_grad=True),
 tensor(-7.83308738, requires_grad=True),
 tensor(-7.80466849, requires_grad=True),
 tensor(-7.77504582, requires_grad=True),
 tensor(-7.7479222, requires_grad=True),
 tensor(-7.72783052, requires_grad=True),
 tensor(-7.71902279, requires_grad=True),
 tensor(-7.72193534, requires_grad=True),
 tensor(-7.47129198, requires_grad=True),
 tensor(-7.4251509, requires_grad=True),
 tensor(-7.39870975, requires_grad=True),
 tensor(-7.38208096, requires_grad=True),
 tensor(-7.37130302, requires_grad=True)]

In [7]:
LiH_adapt_vqe_data = np.column_stack((r_array, E_R))
file_name = 'LiH_adapt_vqe_data_e_3_active_electron_4.txt'
np.savetxt(file_name, LiH_adapt_vqe_data, delimiter=',', header='r_array, E_R', comments='')